In [ ]:
import time
import datetime
import pandas as pd
from google.api_core import datetime_helpers
from google.cloud import bigquery

In [ ]:
project_id = '' #List project
newest_dataset = '' #Newest dataset will all data
recovery_dataset = '' #Dataset to recover based on table names in existing dataset

In [ ]:
def find_tables(project_id="", dataset=""):
    query="""SELECT table_name
        FROM
        {}.INFORMATION_SCHEMA.TABLES
        where table_catalog = '{}'
        and table_schema = '{}'
        """.format(dataset,project_id, dataset)
    df = pd.read_gbq(query=query,project_id=project_id, dialect="standard")
    return list(df["table_name"])

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [ ]:
tables = find_tables(project_id, newest_dataset)

In [ ]:
tables

In [ ]:
previous_date = "08-04-2021"
previous_date = previous_date.replace("-","")

In [ ]:
newest_date = "12-04-2021"
newest_date = newest_date.replace("-","")

In [ ]:
tables = [tbl.replace(newest_date,previous_date) for tbl in tables]

In [ ]:
# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

In [ ]:
tables_to_recover = tables
#snapshot_epoch = int(unix_time_millis(datetime.datetime(2021,2,2)))
epoch = 1617855600#Use the link to get it http://www.vk2zay.net/calculators/epochTimeConverter.php   and select Australia/Sydney
snapshot_epoch = epoch*1000

In [ ]:
#TODO(developer): Choose a table to recover.
for table in tables_to_recover:
    table_id = f"{project_id}.{recovery_dataset}.{table}"

    # TODO(developer): Choose a new table ID for the recovered table data.
    recovered_table_id = f"{project_id}.{recovery_dataset}.{table}"

    # TODO(developer): Choose an appropriate snapshot point as epoch
    # milliseconds. For this example, we choose the current time as we're about
    # to delete the table immediately afterwards.


    #use the below snapshot epoch if want to goi back to specific date
    
    snapshot_table_id = "{}@{}".format(table_id, snapshot_epoch)
    
        # Construct and run a copy job.
    try:
        job = client.copy_table(
            snapshot_table_id,
            recovered_table_id,
            # Must match the source and destination tables location.
            location="australia-southeast1",
        )  # Make an API request.

        job.result()  # Wait for the job to complete.

        print(
            "Copied data from deleted table {} to {}".format(table_id+f"@{snapshot_epoch}", recovered_table_id)
        )
    except Exception as e:
        print(e)